In [1]:
import pandas as pd
import os
import requests as re
import json
import sqlite3
from sqlite3 import Error
from pprint import pprint

In [2]:
#set the keys
wwo_key = os.environ["WWO_API_KEY"]

In [3]:
def wwo_histweather(IATA_code, month, year):
    
    start_date = f'{year}-{month:02}-01'
    
    if month == 2:
        end_date = f'{year}-{month:02}-28'
    elif month in [1, 3, 5, 7, 8, 10, 12]:
         end_date = f'{year}-{month:02}-31'
    else:
        end_date = f'{year}-{month:02}-30'
    

    url_base = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx"
    url = url_base+"?key="+wwo_key+"&q="+IATA_code+"&format=json"+"&date="+start_date+"&enddate="+end_date+"&tp=24"

    res = re.get(url)
    res.raise_for_status()
    
    return res.json()

In [4]:
def wwo_hwparser(monthly_weather, IATA_code):
    
    
    weather_info_list = []
     
    
    for day in monthly_weather['data']['weather']:
        
        weather_info = {'IATA_code': f"\'{IATA_code}\'"}
        
        weather_info['date'] = f"\'{day['date']}\'"
        weather_info['min_temp'] = day['mintempC']
        weather_info['max_temp'] = day['maxtempC']
        weather_info['avg_temp'] = day['avgtempC']
        weather_info['total_snow_cm'] = day['totalSnow_cm']
        weather_info['windspeed_kmph'] = day['hourly'][0]['windspeedKmph']
        weather_info['precipitation_mm'] = day['hourly'][0]['precipMM']
        weather_info['humidity'] = day['hourly'][0]['humidity']
        weather_info['visibility'] = day['hourly'][0]['visibility']
        weather_info['cloud_cover'] = day['hourly'][0]['cloudcover']
        weather_info['heat_index_c'] = day['hourly'][0]['HeatIndexC']
        weather_info['wind_chill_c'] = day['hourly'][0]['WindChillC']
        weather_info['wind_gust_kmph'] = day['hourly'][0]['WindGustKmph']
        weather_info['feels_like'] = day['hourly'][0]['FeelsLikeC']
        weather_info['weather_description'] = f"\'{day['hourly'][0]['weatherDesc'][0]['value']}\'"
        weather_info['pressure'] = day['hourly'][0]['pressure']
        
        weather_info_list.append(weather_info)
    
    return weather_info_list

In [5]:
def wwo_years(IATA_code):
    
    year = 2018
    weather_years = []
    
    try:
        for n in range(0,25):

            month = (n%12)+1

            # Extract weather data for a month for an airport 
            monthly_weather = wwo_histweather(IATA_code, month, year)
            # Parse data
            weather_info_list = wwo_hwparser(monthly_weather, IATA_code)

            weather_years.append(weather_info_list)

            if month == 12:
                year += 1
    
    except re.HTTPError:
        print("HTTPError was raised!")
    
    return weather_years  #list of lists of dicts: month --> days 

# Insert into SQLite database

In [6]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [7]:
connection = create_connection("data/weather_db.sqlite")

Connection to SQLite DB successful


In [7]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [9]:
create_users_table = """
CREATE TABLE IF NOT EXISTS weather (
  weather_id INTEGER PRIMARY KEY AUTOINCREMENT,
  IATA_code TEXT,
  date TEXT,
  min_temp INTEGER,
  max_temp INTEGER,
  avg_temp INTEGER,
  total_snow_cm REAL,
  windspeed_kmph INTEGER,
  precipitation_mm REAL,
  humidity INTEGER,
  visibility INTEGER,
  cloud_cover INTEGER,
  heat_index_c INTEGER,
  wind_chill_c INTEGER,
  wind_gust_kmph INTEGER,
  feels_like INTEGER,
  weather_description TEXT,
  pressure REAL
);
"""
execute_query(connection, create_users_table)

Query executed successfully


In [8]:
def insert_records(database, weather_years):
    
    weather_base = """
    INSERT INTO
      weather (IATA_code, date, min_temp, max_temp, avg_temp, total_snow_cm, windspeed_kmph, precipitation_mm, humidity, visibility, 
              cloud_cover, heat_index_c, wind_chill_c, wind_gust_kmph, feels_like, weather_description, pressure)
    VALUES

    """
    query = weather_base
    insert_value = "("

    for month in weather_years:
        
        if insert_value != "(":
            query += insert_value
            query = query[:-2] + "),"
            insert_value = "("
                    
        for day in month:
            
            if insert_value != "(":
                insert_value = insert_value[:-2] + '''),
('''
            
            for value in day.values():
                
                insert_value += f"{value}, "
                
                

    query += insert_value
    query = query[:-2] + ");"
    
    execute_query(database, query)
    return query

# MAIN

In [28]:
IATA_code = 'DTW'
# month = 1
# year = 2018

# res_JSON = wwo_histweather(IATA_code, month, year)
# month_weather = wwo_hwparser(res_JSON,IATA_code)
weather_years = wwo_years(IATA_code)

In [29]:
query = insert_records(connection, weather_years)

Query executed successfully
